<a href="https://colab.research.google.com/github/Cactice/bert-ja-experiment/blob/master/first_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone --recurse-submodules https://github.com/yoheikikuta/bert-japanese
!pip install keras-bert
!pip install gdown
!pip install sentencepiece
!git clone https://github.com/Cactice/bert-ja-experiment

Cloning into 'bert-japanese'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 2 (delta 0), pack-reused 160
Receiving objects: 100% (164/164), 218.44 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Submodule 'bert' (https://github.com/google-research/bert.git) registered for path 'bert'
Submodule 'wikiextractor' (https://github.com/attardi/wikiextractor.git) registered for path 'wikiextractor'
Cloning into '/content/bert-japanese/bert'...
remote: Enumerating objects: 340, done.        
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340        
Receiving objects: 100% (340/340), 310.71 KiB | 10.36 MiB/s, done.
Resolving deltas: 100% (186/186), done.
Cloning into '/content/bert-japanese/wikiextractor'...
remote: Enumerating objects: 613, done.        
remote: Total 613 (delta 0), reused 0 (delta 0), pack-reused 613        
Receiving objects: 10

In [0]:
  # https://drive.google.com/file/d/11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In/view?usp=sharing
  # https://drive.google.com/file/d/1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS/view?usp=sharing
  # https://drive.google.com/file/d/1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN/view?usp=sharing
  # https://drive.google.com/file/d/1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-/view?usp=sharing
  # https://drive.google.com/file/d/1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB/view?usp=sharing
  # https://drive.google.com/file/d/1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY/view?usp=sharing
  # https://drive.google.com/file/d/1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1/view?usp=sharing


[Errno 2] No such file or directory: 'bert-japanese/model'
/content


In [0]:
!gdown https://drive.google.com/uc?id=11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In
!gdown https://drive.google.com/uc?id=1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS
!gdown https://drive.google.com/uc?id=1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN
!gdown https://drive.google.com/uc?id=1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-
!gdown https://drive.google.com/uc?id=1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB
!gdown https://drive.google.com/uc?id=1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY
!gdown https://drive.google.com/uc?id=1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1

Downloading...
From: https://drive.google.com/uc?id=11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In
To: /content/graph.pbtxt
11.0MB [00:00, 96.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS
To: /content/jawiki-20181220-pages-articles-multistream.xml.bz2
2.91GB [00:27, 105MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN
To: /content/model.ckpt-1400000.data-00000-of-00001
1.33GB [00:23, 56.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-
To: /content/model.ckpt-1400000.index
100% 9.35k/9.35k [00:00<00:00, 9.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB
To: /content/model.ckpt-1400000.meta
4.84MB [00:00, 75.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY
To: /content/wiki-ja.model
100% 806k/806k [00:00<00:00, 52.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uzPpW38L

In [0]:
%cd /content
!ls 

/content
bert-ja-experiment				    model.ckpt-1400000.index
bert-japanese					    model.ckpt-1400000.meta
graph.pbtxt					    sample_data
jawiki-20181220-pages-articles-multistream.xml.bz2  wiki-ja.model
model.ckpt-1400000.data-00000-of-00001		    wiki-ja.vocab


graph.pbtxt					    model.ckpt-1400000.meta
jawiki-20181220-pages-articles-multistream.xml.bz2  wiki-ja.model
model.ckpt-1400000.data-00000-of-00001		    wiki-ja.vocab
model.ckpt-1400000.index


In [0]:
import sys
sys.path.append('modules')
from keras_bert import load_trained_model_from_checkpoint

config_path = 'bert-ja-experiment/bert_config.json'
# `model.ckpt-1400000` のように拡張子を付けないのがポイントです。
checkpoint_path = 'model.ckpt-1400000'

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path)
bert.summary()

Using TensorFlow backend.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
import pandas as pd
import sentencepiece as spm
from keras import utils
import logging
import numpy as np

maxlen = 512
bert_dim = 768

sp = spm.SentencePieceProcessor()
sp.Load('wiki-ja.model')

def _get_vector(feature):
    common_seg_input = np.zeros((1, maxlen), dtype = np.float32)
    indices = np.zeros((1, maxlen), dtype = np.float32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        try:
            indices[0, t] = sp.piece_to_id(token)
        except:
            logging.warn(f'{token} is unknown.')
            indices[0, t] = sp.piece_to_id('<unk>')
    vector =  bert.predict([indices, common_seg_input])[0]

    return vector

def _load_labeldata(train_dir, test_dir):
    train_features_df = pd.read_csv(f'{train_dir}/features.csv')
    train_labels_df = pd.read_csv(f'{train_dir}/labels.csv')
    test_features_df = pd.read_csv(f'{test_dir}/features.csv')
    test_labels_df = pd.read_csv(f'{test_dir}/labels.csv')
    label2index = {k: i for i, k in enumerate(train_labels_df['label'].unique())}
    index2label = {i: k for i, k in enumerate(train_labels_df['label'].unique())}
    class_count = len(label2index)
    train_labels = utils.np_utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    test_label_indices = [label2index[label] for label in test_labels_df['label']]
    test_labels = utils.np_utils.to_categorical(test_label_indices, num_classes=class_count)

    train_features = []
    test_features = []

    for feature in train_features_df['feature']:
        train_features.append(_get_vector(feature))
    for feature in test_features_df['feature']:
        test_features.append(_get_vector(feature))

    print(f'Trainデータ数: {len(train_features_df)}, Testデータ数: {len(test_features_df)}, ラベル数: {class_count}')

    return {
        'class_count': class_count,
        'label2index': label2index,
        'index2label': index2label,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'test_label_indices': test_label_indices,
        'train_features': np.array(train_features),
        'test_features': np.array(test_features),
        'input_len': maxlen
    }

In [0]:
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras import Input, Model

def _create_model(train_features):
    class_count = 4

    input_tensor = Input(train_features[0].shape)
    x1 = Bidirectional(LSTM(356))(input_tensor)
    output_tensor = Dense(class_count, activation='softmax')(x1)

    model = Model(input_tensor, output_tensor)
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['mae', 'mse', 'acc'])

    return model

In [16]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

model_filename = 'knbc-check-bert_v2.model'

model = _create_model(data['train_features'])
model.summary()
history = model.fit(data['train_features'],
          data['train_labels'],
          epochs = 100,
          batch_size = 128,
          validation_data=(data['test_features'], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              EarlyStopping(patience=5, monitor='val_acc', mode='max'),
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])

NameError: ignored